# Data Understanding

In this file we will take a first look at the data we have. We will try to understand the data and get a first impression of it. We will also try to get a first idea of what we can do with the data.

Afterwards we will clean and prepare the data in [02_preparation.ipynb](02_preparation.ipynb). Afterwards we will plot the data in [03_plotting.ipynb](03_plotting.ipynb).


## Config and Imports


In [ ]:
from pathlib import Path
import json

import numpy as np

from studienarbeit.utils.load import EDataTypes, Load

In [ ]:
file_name = "prep_tweets_sent_full.parquet"
data_type = EDataTypes.TWEETS

load = Load(data_type=data_type)

In [ ]:
df = load.load_dataframe(
    "tweets.parquet", columns=["screen_name", "created_at", "is_retweet", "text", "party", "birthyear", "gender"]
)

## Analyse the Data


In [ ]:
# Print the initial shape of the dataframe
df.shape

In [ ]:
# Replace empty string-array conversions from r to python
for col in df.columns:
    df[col] = df[col].apply(
        lambda x: None if x == "" or x == "NA" or x == "NA, NA" or x == "NA, NA, NA, NA, NA, NA, NA, NA" else x
    )

In [ ]:
# Print the count and distribution before any preprocessing
print(f"Shape before dropping na: {df[df['party'] != 'Parteilos'].shape}")
print(f"\nParty distribution before preprocessing: \n{df[df['party'] != 'Parteilos']['party'].value_counts()}")

In [ ]:
# Count the number of retweets per party
df[df["is_retweet"] == "TRUE"]["party"].value_counts()

In [ ]:
# Calculate the mean number of tweets (id) per screen_name grouped by party
df.groupby("party")["screen_name"].value_counts().groupby("party").describe()

In [ ]:
# Count the number of unique users
df["screen_name"].nunique()

In [ ]:
# Check for missing values
df.isna().sum()

In the cell above we can see that there are about 11k missing values in the `text` column. Regarding the `is_retweet` column, about 3k entries have missing values.

Following we will delete the rows.


In [ ]:
df = df.dropna(subset=["text", "is_retweet"])

In [ ]:
# Check how many unique values are in the columns
df.nunique()

In [ ]:
# Clean duplicated rows (some tweets seem to be scraped twice at different days)
df = df.drop_duplicates(
    subset=["screen_name", "is_retweet", "text", "party", "birthyear", "gender"], keep="last"
)

In [ ]:
df["party"].value_counts()

In [ ]:
df.groupby("gender")["screen_name"].nunique()

In [ ]:
convert_dict = {
    "screen_name": "string[pyarrow]",
    "created_at": "datetime64[ns]",
    "is_retweet": "category",
    "text": "string[pyarrow]",
    "party": "category",
    "birthyear": "datetime64[ns]",
    "gender": "category",
}

In [ ]:
df = df.astype(convert_dict)

In [ ]:
df.info(verbose=True, memory_usage="deep")

In [ ]:
df.describe(include="all", datetime_is_numeric=True)

In [ ]:
df.head()

In [ ]:
load.save_dataframe(df, "tweets_understanding.parquet")

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Profiling Report")
profile.to_file("01_understanding.html")